### Ingest lap_times.json file

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [ ]:
v_data_source = "" #Provided by Data Factory

In [ ]:
%run configuration

In [ ]:
%run common_functions

In [1]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lit

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 3, Finished, Available)

In [2]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 4, Finished, Available)

In [8]:
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv(f"{bronze_folder_path}/lap_times")
display(lap_times_df)

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, fd7c24c3-fd78-4353-aa3d-02a50a43898e)

##### Step 2 - Rename columns and add new columns
1. Rename driverId and raceId
1. Add ingestion_date with current timestamp

In [ ]:
final_df = add_ingestion_date(lap_times_df)

In [9]:
final_df = final_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("data_source", lit(v_data_source))
display(final_df)

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 9c445183-4d29-4751-b94f-3a25ba0e1786)

##### Step 3 - Write to output to processed container in parquet format

In [6]:
final_df.write.mode('overwrite').parquet(f"{silver_folder_path}/lap_times")

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 8, Finished, Available)

In [7]:
df_parquet = spark.read.parquet(f"{silver_folder_path}/lap_times")
display(df_parquet)

StatementMeta(, f78f61f7-9ec3-4f23-88e3-54652143c4a6, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, d5b5d443-0629-4012-bdaf-b95cafeec5e3)